In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import openpyxl
import re
import os
from variableUtils import *
import variableUtils
from Utils import *
from ClassUtils import *
from pprint import pprint
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from reportlab.lib.pagesizes import letter, landscape, A4, A3
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, PageBreak, Paragraph, Spacer, Image
from reportlab.lib import colors
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.platypus import Paragraph, Spacer, KeepTogether, KeepInFrame
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import io
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule
import PIL
import json
import textwrap
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(sns.__version__)


(841.68, 1190.8799999999999)
0.13.2


In [23]:
# workbookpath = '2024\DCD\Postgraduate Clinical Evaluation Paeds_March 14, 2025_13.24.xlsx'
# workbookpath = '2025\DCD\Paeds\Postgraduate Clinical Evaluation Paeds - 2025_April 10, 2025_12.29.xlsx'
workbookpath = '2025\DCD\Oral Med\Postgraduate Clinical Evaluation - DCD in Oral Medicine_May 15, 2025_14.31.xlsx'
workbookpath =  '2025\DCD\Prostho\Postgraduate Clinical Evaluation - DCD in Prosthodontics_May 15, 2025_14.32.xlsx'
folder, file, ext = getFolderandFileName(workbookpath)
df = pd.read_excel(workbookpath)
fullnamedict = {df.columns[i]: df.iloc[0, i] for i in range(len(df.columns))}
df = df[1:]
colName = 'Student Name'
# mccols are with Q\d+_\d+ pattern
mcCols = [col for col in df.columns if re.match(r'Q\d+_\d+', col)]
mcColIdentifier = 'Q9'
# remove 
generalCols = [col for col in mcCols if mcColIdentifier in col]
mcCols = [col for col in mcCols if mcColIdentifier not in col]
print(mcCols)
colDate = 'Date'
extraCols = ['Time management', 'Entrustment scale']

commentCol1 = 'Q10'
commentCol2 = 'Critical incident'
finalgradedict = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':5, 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':4, 
                  'At expected level of performance considering stage of training: H3 (65-69)':3, 'Barely meets expected level of performance considering stage of training: P (50-64)':2,
                  'Does not meet expected level of performance considering stage of training: N (0-49)':1, 'No basis for evaluation':np.nan, 'Clinical rotation not completed':np.nan, 'Not applicable':np.nan}
colorlegendfinal = {'Far exceeds expected level of performance considering stage of training: H1 (80-100)':'#4c72b0', 'Exceeds expected level of performance considering stage of training: H 2A (75-79) and B (70-74)':'#55a868',
                    'At expected level of performance considering stage of training: H3 (65-69)':'#e07a5f', 'Barely meets expected level of performance considering stage of training: P (50-64)':'#f9c74f',
                    'Does not meet expected level of performance considering stage of training: N (0-49)':'#993f14', 'Clinical rotation not completed':'gray'}
# colFinalGrade = 'Q22'
# df[colFinalGrade] = df[colFinalGrade].replace(finalgradedict)
# df[colFinalGrade] = df[colFinalGrade].astype(float)
mcgradedict = {'Completed with substantial assistance':1, 'Completed with minimal assistance':2, 'Completed independently (competent)':3, 'No basis for evaluation': np.nan}
generalgradedict = {'ALWAYS': 4, 'OFTEN': 3, 'SOMETIMES': 2, 'RARELY': 1, 'NEVER': 0, 'Not graded': np.nan}
# create color legend dictionary
colorlegendmc = {'Completed with substantial assistance': '#e07a5f', 'Completed with minimal assistance': '#55a868', 'Completed independently (competent)': '#4c72b0', 'No basis for evaluation': 'gray',
              }
colorlegendgeneral =  {'ALWAYS': '#4c72b0', 'OFTEN': '#55a868', 'SOMETIMES': '#e07a5f', 'RARELY': '#f9c74f', 'NEVER': '#993f14', 'Not graded': 'gray'}
reversemcgradedict = {v:k for k, v in mcgradedict.items()}
referencedf = pd.DataFrame(reversemcgradedict.items(), columns=['Grade', 'Description'])
# drop nan values from grade column
referencedf.dropna(inplace=True)
referencedf['Grade'] = referencedf['Grade'].astype(int)
referencedf.sort_values(by='Grade', inplace=True)
referencetable = createTable(referencedf, 'Score reference', [1, 5], customTextCols=[1])
df[mcCols] = df[mcCols].replace(mcgradedict)
df[colDate] = pd.to_datetime(df[colDate], format = 'mixed')
df.sort_values(by=[colName, colDate], inplace=True)
df[colDate] = df[colDate].dt.date
df[colDate] = df[colDate].astype(str)
df[mcCols] = df[mcCols].astype(float)

# count the number of each scores for each student
df['#3'] = df[mcCols].apply(lambda x: x.value_counts().get(3, 0), axis=1)
df['#2'] = df[mcCols].apply(lambda x: x.value_counts().get(2, 0), axis=1)
df['#1'] = df[mcCols].apply(lambda x: x.value_counts().get(1, 0), axis=1)
df['#nan'] = df[mcCols].apply(lambda x: pd.isna(x)).sum(axis=1)

df[generalCols] = df[generalCols].replace(generalgradedict)
# display(df.head())

students = df[colName].unique()

# for student in students:
def getReport(student):
    elements = []
    elements.append(Paragraph(f'{student}', style = headingStyle))
    elements.append(Spacer(1, 36))
    # elements.append(Paragraph(f'Number of forms: {len(df[df[colName] == student])}', style = subheadingStyle))
    # elements.append(Spacer(1, 12))
    elements.append(Paragraph('Marking Checklists', style = subheadingStyle))
    elements.append(Spacer(1, 12))
    elements.append(referencetable)
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = subsubheadingStyle))
    elements.append(Spacer(1, 24))
    # plot time series of scores for each mc question ignoring the nan values
    for col in mcCols:
        fig, ax = plt.subplots(1,2, figsize=(30, 5))
        xy = studentdf[[colDate, col]].dropna()
        # aggregate the scores for each date by taking the mean
        xy = xy.groupby(colDate).mean().reset_index()
        display(xy.head())
        ax[0].plot(xy[colDate], xy[col], marker='o', color='blue')
        title = fullnamedict[col].split(' - ')[-1]
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90, ha='right')
        ax[0].yaxis.set_major_locator(plt.MaxNLocator(integer=True))
        ax[0].set_title('Time Series of Scores')
        plt.grid('--', alpha=0.5)
        ax[0].set_ylim(0, 3.5)

        valuecounts = studentdf[col].value_counts(dropna=False)
        # valuecounts.index = valuecounts.index.astype(int)
        valuecounts.index = [k for k, v in mcgradedict.items() if v in valuecounts.index]
        total = valuecounts.sum()
        colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
        ax[1].pie(valuecounts, labels=valuecounts.index, autopct= lambda x: autopct(x, total), colors=colors)
        ax[1].set_title('Distribution of Scores')
        wrappedTitle = "\n".join(textwrap.wrap(fullnamedict[col].split(' - ')[-1], width=80))  # Adjust width as needed
        # plt.suptitle(wrappedTitle)
        img = addPlotImage(fig, 0.9)
        smallsubsubheadingStyle = ParagraphStyle('Heading3', parent=styles['Heading3'], fontSize=13, alignment=1)
        elements.append(KeepTogether([Paragraph(wrappedTitle, style = smallsubsubheadingStyle), Spacer(1, 12), img]))
        # elements.append(img)
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    elements.append(PageBreak())
    # plot time series of scores for each general question ignoring the nan values
    for col in generalCols:
        fig, ax = plt.subplots(1,2, figsize=(30, 5))
        xy = studentdf[[colDate, col]].dropna()
        # aggregate the scores for each date by taking the mean
        xy = xy.groupby(colDate).mean().reset_index()
        display(xy.head())
        ax[0].plot(xy[colDate], xy[col], marker='o', color='blue')
        title = fullnamedict[col].split(' - ')[-1]
        ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation=90, ha='right')
        ax[0].yaxis.set_major_locator(plt.MaxNLocator(integer=True))
        ax[0].set_title('Time Series of Scores')
        plt.grid('--', alpha=0.5)
        ax[0].set_ylim(0, 4.5)

        valuecounts = studentdf[col].value_counts(dropna=False)
        # valuecounts.index = valuecounts.index.astype(int)
        valuecounts.index = [k for k, v in generalgradedict.items() if v in valuecounts.index]
        total = valuecounts.sum()
        colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
        ax[1].pie(valuecounts, labels=valuecounts.index, autopct= lambda x: autopct(x, total), colors=colors)
        ax[1].set_title('Distribution of Scores')
        wrappedTitle = "\n".join(textwrap.wrap(fullnamedict[col].split(' - ')[-1], width=80))
        # plt.suptitle(wrappedTitle)
        img = addPlotImage(fig, 0.9)
        smallsubsubheadingStyle = ParagraphStyle('Heading3', parent=styles['Heading3'], fontSize=13, alignment=1)
        elements.append(KeepTogether([Paragraph(wrappedTitle, style = smallsubsubheadingStyle), Spacer(1, 12), img]))
        # elements.append(img)
        elements.append(Spacer(1, 24))
        plt.close(fig)

    elements.append(PageBreak())

    elements.append(Paragraph('Final Grades', style = headingStyle))
    elements.append(Spacer(1, 36))
    reversefinalgradedict = {v:k for k, v in finalgradedict.items()}
    refdf2 = pd.DataFrame(reversefinalgradedict.items(), columns=['Grade', 'Description'])
    refdf2.dropna(inplace=True)
    refdf2['Grade'] = refdf2['Grade'].astype(int)
    # drop nan values from grade column

    refdf2.sort_values(by='Grade', inplace=True)
    refTable2 = createTable(refdf2, 'Final Grade Reference', [1, 5], customTextCols=[1])
    elements.append(refTable2)
    elements.append(Spacer(1, 24))
    # plot time series of final grades
    fig, ax = plt.subplots(1,1, figsize=(30, 5))
    xy = studentdf[[colDate, colFinalGrade]].dropna()
    xy = xy.groupby(colDate).mean().reset_index()     # aggregate the scores for each date by taking the mean
    ax.plot(xy[colDate], xy[colFinalGrade])
    title = 'Final Grade Time Series'
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))     # integer ticks on y axis
    ax.set_ylim(0, 5)
    ax.set_title(title)
    plt.grid('--', alpha=0.5)
    img = addPlotImage(fig, 0.9)
    elements.append(img)
    plt.close(fig)
    elements.append(Spacer(1, 36))
    
    # pie chart of final grades
    fig, ax = plt.subplots(1,1, figsize=(10, 10))
    xy = studentdf[colFinalGrade].dropna()
    xy = xy.value_counts()
    total = xy.sum()
    xy = xy.sort_index()
    xy.index = [k for k, v in finalgradedict.items() if v in xy.index]     # labels based on finalgradedict
    ax.pie(xy, labels=xy.index, autopct= lambda x: autopct(x, total))
    title = 'Final Grade Distribution'
    ax.set_title(title)
    
    img = addPlotImage(fig, 0.9)
    elements.append(img)
    plt.close(fig)

    # time series of number of each score
    fig, ax = plt.subplots(1, 1, figsize=(30, 10))
    xy = studentdf[[colDate, '#3', '#2', '#1', '#nan']].dropna()
    xy = xy.groupby(colDate).sum().reset_index()
    xy.set_index(colDate, inplace=True)
    xy.plot(kind='bar', stacked=True, color=['blue', 'green', 'orange', 'gray'], ax=ax)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')
    ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))
    ax.set_title('Stacked Bar Chart of Number of Each Score')
    ax.legend(title='Scores')
    plt.grid('--', alpha=0.5)
        # ax[i].set_ylim(0, 5)
    img = addPlotImage(fig, 0.9)
    elements.append(KeepTogether([Paragraph('Counts of Each Score', style = subheadingStyle), Spacer(1, 12), img]))
    plt.close(fig)
    elements.append(PageBreak())

    savefolder = f'{folder}/studentReports'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)

def chunkList(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i+size]


def createPieCharts(mcCols, n, mcgradedict, colorlegend, elements, studentdf, fullnamedict):
    for groupIndex, mcChunk in enumerate(chunkList(mcCols, n)):
        fig, axes = plt.subplots(3, 2, figsize=(figSize[0], figSize[1]))
        fig.subplots_adjust(hspace=0.4, wspace=0.3)
        axes = axes.flatten()
        for idx, col in enumerate(mcChunk):
            ax = axes[idx]
            yTitle = 1.3
            faceColor = '#f2f2f2'  # background shading
            # add frame
            framePadding = 0.02
            bbox = ax.get_position()
            # Calculate extended box
            x0 = bbox.x0 - 2 * framePadding
            y0 = bbox.y0 - framePadding
            width = bbox.width + 4 * framePadding
            height = bbox.height + (yTitle - 1.23) + framePadding  # 1.0 is top of axes

            # Draw rectangle in figure space
            rect = Rectangle(
                (x0, y0), width, height,
                linewidth=1,
                edgecolor='black',
                facecolor=faceColor,
                transform=fig.transFigure,
                zorder=-5
            )
            fig.patches.append(rect)


            valuecounts = studentdf[col].value_counts(dropna=False)   
            display(valuecounts) 
            newkeys = []
            for v in valuecounts.index:
                for k, val in mcgradedict.items():
                    if (pd.isna(v) and pd.isna(val)) or (v == val):
                        newkeys.append(k)
                        break
            print(newkeys)

            valuecounts.index = newkeys
            total = valuecounts.sum()
            display(valuecounts)
            colors = [colorlegend.get(k, 'black') for k in valuecounts.index]
            
            # create pie without labels
            wedges, _, autotexts = ax.pie(
                valuecounts,
                labels=None,
                autopct=lambda x: autopct(x, total),
                colors=colors,
                textprops={'fontsize': 7}
            )
            

            title = textwrap.fill(fullnamedict[col].split(' - ')[-1], width=55)  # Adjust width as needed
            # ax.set_title(title, fontsize=9)
            ax.text(0.5, yTitle, title, ha='center', va='top', fontsize=8, transform=ax.transAxes, wrap=True)


            # Add legend to bottom-left of figure
            legendHandles = [
                plt.Line2D([0], [0], marker='o', color='w', label=label,
                        markerfacecolor=color, markersize=8)
                for label, color in colorlegend.items()
            ]

            fig.legend(
                handles=legendHandles,
                loc='upper left',
                bbox_to_anchor=(0.09, 1.03),
                fontsize=8
            )

        # Hide unused subplots
        for j in range(len(mcChunk), n):
            axes[j].axis('off')


        # Save or display
        img = addPlotImage(fig, 0.9)
        # elements.append(KeepTogether([Paragraph(f"Batch {groupIndex+1} Pie Charts", smallsubsubheadingStyle), Spacer(1, 12), img]))
        elements.append(KeepTogether([img]))
        elements.append(Spacer(1, 24))
        plt.close(fig)
    
    # elements.append(PageBreak())
def getReportType2(student): # only plot the pie charts for each mc question, general question and final grade
    elements = []
    elements.append(Paragraph(f'{student}', style = subheadingStyle))
    elements.append(Spacer(1, 24))
    studentdf = df[df[colName] == student]
    display(studentdf.head())
    elements.append(Paragraph(f'Number of forms: {len(studentdf)}', style = tableTextStyleLarge))
    elements.append(Spacer(1, 24))
    # plot mc question ignoring the nan values
    n = 6
    createPieCharts(mcCols, n, mcgradedict, colorlegendmc, elements, studentdf, fullnamedict)
    elements.append(PageBreak())

    # plot general question ignoring the nan values
    createPieCharts(generalCols, n, generalgradedict, colorlegendgeneral, elements, studentdf, fullnamedict)
    elements.append(PageBreak())


    # # plot final grade ignoring the nan values
    # elements.append(Paragraph('Final Grades', style = subsubheadingStyle))
    # elements.append(Spacer(1, 24))
    # reversefinalgradedict = {v:k for k, v in finalgradedict.items()}
    # refdf2 = pd.DataFrame(reversefinalgradedict.items(), columns=['Grade', 'Description'])
    # refdf2.dropna(inplace=True)
    # refdf2['Grade'] = refdf2['Grade'].astype(int)
    # # drop nan values from grade column

    # refdf2.sort_values(by='Grade', inplace=True)
    # refTable2 = createTable(refdf2, 'Final Grade Reference', [1, 5], customTextCols=[1])
    # # elements.append(refTable2)
    # # elements.append(Spacer(1, 24))
    # # plot time series of final grades
    # fig, ax = plt.subplots(1,1, figsize=(figSize[0], figSize[1]/5))
    # xy = studentdf[[colDate, colFinalGrade]].dropna()
    # xy = xy.groupby(colDate).mean().reset_index()     # aggregate the scores for each date by taking the mean
    # ax.plot(xy[colDate], xy[colFinalGrade])
    # title = 'Average Time Series'
    # ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right', fontsize=6)
    # ax.yaxis.set_major_locator(plt.MaxNLocator(integer=True))     # integer ticks on y axis
    # ax.set_ylim(0, 5.4)
    # ax.set_title(title, fontsize=10)
    # plt.grid('--', alpha=0.5)
    # img = addPlotImage(fig, 0.9)
    # # elements.append(img)
    # plt.close(fig)
    # # elements.append(Spacer(1, 24))
    
    # # pie chart of final grades
    # fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
    # xy = studentdf[colFinalGrade].dropna()
    # xy = xy.value_counts()
    # total = xy.sum()
    # xy = xy.sort_index()
    # xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
    # labels = [textwrap.fill(k, width=100) for k in xy.index]
    # ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels, colors=[colorlegendfinal.get(k, 'gray') for k in xy.index])
    # # Add legend to bottom-left of figure
    # # legendHandles = [
    # #     plt.Line2D([0], [0], marker='o', color='w', label=label,
    # #             markerfacecolor=color, markersize=6)
    # #     for label, color in colorlegendfinal.items()
    # # ]

    # # fig.legend(
    # #     handles=legendHandles,
    # #     loc='upper left',
    # #     bbox_to_anchor=(0.09, 0.03),
    # #     fontsize=6
    # # )
    # title = 'Final Grade Distribution'
    # ax.set_title(None)
    # img = addPlotImage(fig, 0.9)
    # elements.append(img)
    # plt.close(fig)
    # elements.append(Spacer(1, 24))
    
    # create pie charts for extra columns similar to final grades
    for col in extraCols:
        fig, ax = plt.subplots(1,1, figsize=(figSize[0]*0.6, figSize[0]*0.6))
        xy = studentdf[col].dropna()
        xy = xy.value_counts()
        total = xy.sum()
        xy = xy.sort_index()
        # xy.index =  xy.index.map(reversefinalgradedict)    # labels based on finalgradedict
        labels = [textwrap.fill(k, width=100) for k in xy.index]
        ax.pie(xy, autopct= lambda x: autopct(x, total),textprops={'fontsize': 11}, labels = labels)
        title = f'{col} Distribution'
        ax.set_title(None)
        img = addPlotImage(fig, 0.9)
        elements.append(KeepTogether([Paragraph(title, style = subheadingStyle), Spacer(1, 12), img]))
        plt.close(fig)
        elements.append(Spacer(1, 24))
    
    # add comments of Q20 and Q21 as table
    commentsdfq20 = studentdf[['Staff', commentCol1]].dropna()
    commentsdfq20.rename(columns={commentCol1: 'Comments'}, inplace=True)
    display(commentsdfq20.head())
    commentstableq20 = createTable(commentsdfq20, f'{fullnamedict[commentCol1]}', [1, 5], customTextCols=[0, 1], titleStyle=variableUtils.smallsubsubheadingStyleC, tableTextStyle=tableTextStyleSmall)
    elements.append(commentstableq20)
    elements.append(Spacer(1, 24))
    commentsdfq21 = studentdf[['Staff', commentCol2]].dropna()
    commentsdfq21.rename(columns={commentCol2: 'Comments'}, inplace=True)
    display(commentsdfq21.head())
    commentstableq21 = createTable(commentsdfq21, f'{fullnamedict[commentCol2]}', [1, 5], customTextCols=[0, 1], titleStyle=variableUtils.smallsubsubheadingStyleC, tableTextStyle=tableTextStyleSmall)
    elements.append(commentstableq21)
    # elements.append(Spacer(1, 24))

    savefolder = f'{folder}/studentReports Brief'
    os.makedirs(savefolder, exist_ok=True)
    doc= SimpleDocTemplate(f'{savefolder}/{student}.pdf', pagesize=pageSize, rightMargin=rightMargin, leftMargin=leftMargin, topMargin=topMargin, bottomMargin=bottomMargin)
    doc.build(elements)
students = df[colName].unique()
for student in students:
    getReportType2(student)
    # break
# getReport('Grace Wu')
# getReport('Andrew Park')

['Q7_1', 'Q7_2', 'Q8_1', 'Q8_2', 'Q8_3', 'Q8_4', 'Q8_5', 'Q8_6', 'Q8_7', 'Q8_8', 'Q8_9', 'Q8_10']
Creating table for Score reference


,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Date,Student Name,Site,Staff,Q7_1,Q7_2,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q9_1,Q9_2,Q9_3,Q9_4,Q9_5,Q9_6,Q9_7,Q9_8,Q9_9,Q10,Critical incident,CI Details,Time management,Entrustment scale,#3,#2,#1,#nan
1,2025-03-20 10:20:53,2025-03-20 10:26:01,IP Address,128.250.0.47,100,308,True,2025-03-20 10:26:03.113000,R_4VVZepfWjVlesQ9,NaN,NaN,NaN,NaN,-37.8024,144.9659,anonymous,EN,2025-03-20,Alex Ho,Melbourne Dental Clinic,Jaafar Abduo,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,lkjb,Yes,;ioj,Lvl 1: Work not completed in allocated timeframe.,Lvl 3: Student can be trusted to perform task ...,12,0,0,0
13,2025-05-08 23:14:19,2025-05-08 23:19:16,IP Address,122.150.165.11,100,296,True,2025-05-08 23:19:18.067000,R_9ump24pO0WYwxLH,NaN,NaN,NaN,NaN,-37.8186,145.1277,anonymous,EN,2025-05-08,Alex Ho,Royal Dental Hospital Melbourne RDHM,Chen Chen,3.0,3.0,3.0,3.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,Good demonstration of managing pt expectations...,No,NaN,Lvl 2: Reaches a step (or point) in the proced...,Lvl 3: Student can be trusted to perform task ...,5,4,3,0


Q7_1
3.0    2
Name: count, dtype: int64

['Completed independently (competent)']


Completed independently (competent)    2
Name: count, dtype: int64

...

ALWAYS    2
OFTEN     1
Name: count, dtype: int64

,Staff,Comments
14,Chen Chen,Demonstrates a low level of clinical proficien...
22,Jaafar Abduo,Noticeable improvements from last year. Yusof'...
27,John Pearson,Has a comprehensive knowledge of prosthodontic...


Creating table for Reflect on the student's performance across the month - what are some strengths and areas you have identified for improvement.


,Staff,Comments
14,Chen Chen,No
22,Jaafar Abduo,No
27,John Pearson,No


Creating table for Has a clinical incident occurred during this month while you were supervising this student?
